# Pump it Up: Data Mining the Water Table
## DrivenData Competition - Water Pump Status Prediction

### Objetivo del Negocio
Predecir el estado funcional de bombas de agua en Tanzania para optimizar el mantenimiento y asegurar el acceso al agua potable. Este modelo ayudará a:
- Identificar bombas que necesitan reparación antes de fallar completamente
- Optimizar recursos de mantenimiento
- Mejorar la disponibilidad de agua para las comunidades

### Clases a Predecir
- **functional**: La bomba funciona correctamente
- **non functional**: La bomba no funciona
- **functional needs repair**: La bomba funciona pero necesita reparación

## 1. Importación de Librerías y Configuración

In [ ]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Librerías de preprocesamiento
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Modelos
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Métricas
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score

# Balanceo de clases
from imblearn.over_sampling import SMOTE
from collections import Counter

print("✓ Librerías importadas correctamente")

## 2. Carga de Datos

### Instrucciones para obtener los datos:
1. Visitar: https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/
2. Registrarse/Iniciar sesión
3. Descargar los archivos:
   - Training set values (train_values.csv)
   - Training set labels (train_labels.csv)
   - Test set values (test_values.csv)
4. Colocar los archivos en el mismo directorio que este notebook

In [ ]:
# Cargar datos
try:
    train_values = pd.read_csv('train_values.csv')
    train_labels = pd.read_csv('train_labels.csv')
    test_values = pd.read_csv('test_values.csv')
    
    print(f"✓ Datos cargados exitosamente")
    print(f"\nDimensiones del dataset de entrenamiento: {train_values.shape}")
    print(f"Dimensiones de las etiquetas: {train_labels.shape}")
    print(f"Dimensiones del dataset de prueba: {test_values.shape}")
    
    # Combinar datos de entrenamiento con etiquetas
    train_data = train_values.merge(train_labels, on='id')
    print(f"\nDataset completo de entrenamiento: {train_data.shape}")
    
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("\nPor favor, descarga los datos siguiendo las instrucciones anteriores.")

## 3. Exploración Inicial de Datos (EDA)

### 3.1 Vista General del Dataset

In [ ]:
# Información general
print("=" * 80)
print("INFORMACIÓN GENERAL DEL DATASET")
print("=" * 80)
print(f"\nNúmero total de bombas: {len(train_data):,}")
print(f"Número de características: {train_data.shape[1] - 1}")

# Primeras filas
print("\n" + "="*80)
print("PRIMERAS FILAS DEL DATASET")
print("="*80)
display(train_data.head())

# Tipos de datos
print("\n" + "="*80)
print("TIPOS DE DATOS")
print("="*80)
print(train_data.dtypes.value_counts())

# Columnas
print("\n" + "="*80)
print("LISTA DE TODAS LAS CARACTERÍSTICAS")
print("="*80)
for i, col in enumerate(train_data.columns, 1):
    print(f"{i:2d}. {col}")

### 3.2 Análisis de la Variable Objetivo

**Insight de Negocio**: Entender la distribución de estados es crucial para:
- Identificar si existe desbalanceo de clases
- Planificar estrategias de mantenimiento según prevalencia
- Ajustar modelos para manejar clases minoritarias

In [ ]:
# Distribución de la variable objetivo
print("=" * 80)
print("DISTRIBUCIÓN DE ESTADOS DE LAS BOMBAS")
print("=" * 80)

status_counts = train_data['status_group'].value_counts()
status_percentages = train_data['status_group'].value_counts(normalize=True) * 100

status_df = pd.DataFrame({
    'Cantidad': status_counts,
    'Porcentaje': status_percentages.round(2)
})
print(status_df)

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
status_counts.plot(kind='bar', ax=axes[0], color=['green', 'red', 'orange'])
axes[0].set_title('Distribución de Estados de Bombas', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Estado')
axes[0].set_ylabel('Cantidad')
axes[0].tick_params(axis='x', rotation=45)

# Gráfico de pastel
axes[1].pie(status_counts, labels=status_counts.index, autopct='%1.1f%%', 
            colors=['green', 'red', 'orange'], startangle=90)
axes[1].set_title('Proporción de Estados', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Insight: Se observa desbalanceo de clases, especialmente en 'functional needs repair'.")
print("   Será necesario aplicar técnicas de balanceo (SMOTE, class weights) en el modelado.")

### 3.3 Análisis de Valores Faltantes

**Contexto de Negocio**: Los datos faltantes pueden indicar:
- Falta de mantenimiento de registros
- Bombas en áreas remotas con poca supervisión
- Necesidad de mejorar sistemas de información

In [ ]:
# Análisis de valores faltantes y ceros
print("=" * 80)
print("ANÁLISIS DE VALORES FALTANTES Y PROBLEMÁTICOS")
print("=" * 80)

missing_data = pd.DataFrame({
    'Missing_Count': train_data.isnull().sum(),
    'Missing_Percent': (train_data.isnull().sum() / len(train_data) * 100).round(2),
    'Zeros_Count': (train_data == 0).sum(),
    'Zeros_Percent': ((train_data == 0).sum() / len(train_data) * 100).round(2)
})

missing_data = missing_data[
    (missing_data['Missing_Count'] > 0) | (missing_data['Zeros_Count'] > 0)
].sort_values('Missing_Percent', ascending=False)

if len(missing_data) > 0:
    print(missing_data)
    
    # Visualización de valores faltantes
    if missing_data['Missing_Percent'].sum() > 0:
        top_missing = missing_data[missing_data['Missing_Percent'] > 0].head(10)
        
        plt.figure(figsize=(12, 6))
        plt.barh(top_missing.index, top_missing['Missing_Percent'])
        plt.xlabel('Porcentaje de Valores Faltantes')
        plt.title('Top 10 Variables con Valores Faltantes', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
else:
    print("✓ No se detectaron valores faltantes explícitos")

# Detectar valores problemáticos en variables específicas
print("\n" + "="*80)
print("VALORES CERO O VACÍOS EN VARIABLES CLAVE")
print("="*80)

# Coordenadas geográficas
print(f"\nCoordenadas con valor 0:")
print(f"  - longitude = 0: {(train_data['longitude'] == 0).sum():,}")
print(f"  - latitude = 0: {(train_data['latitude'] == 0).sum():,}")

# Variables numéricas importantes
numeric_cols = ['gps_height', 'population', 'construction_year']
for col in numeric_cols:
    if col in train_data.columns:
        zero_count = (train_data[col] == 0).sum()
        zero_pct = (zero_count / len(train_data) * 100)
        print(f"  - {col} = 0: {zero_count:,} ({zero_pct:.2f}%)")

### 3.4 Análisis de Variables Categóricas de Alta Cardinalidad

**Desafío de Negocio**: Variables con muchas categorías únicas pueden:
- Dificultar el entrenamiento de modelos
- Requerir técnicas especiales de encoding
- Necesitar agrupación de categorías poco frecuentes

In [ ]:
# Análisis de cardinalidad
print("=" * 80)
print("ANÁLISIS DE CARDINALIDAD - VARIABLES CATEGÓRICAS")
print("=" * 80)

categorical_cols = train_data.select_dtypes(include=['object']).columns.tolist()
if 'status_group' in categorical_cols:
    categorical_cols.remove('status_group')

cardinality_data = []
for col in categorical_cols:
    unique_count = train_data[col].nunique()
    cardinality_data.append({
        'Variable': col,
        'Categorías_Únicas': unique_count,
        'Muestra': ', '.join(train_data[col].value_counts().head(3).index.astype(str))
    })

cardinality_df = pd.DataFrame(cardinality_data).sort_values(
    'Categorías_Únicas', ascending=False
)
print(cardinality_df.to_string(index=False))

# Clasificar por cardinalidad
high_card = cardinality_df[cardinality_df['Categorías_Únicas'] > 100]
medium_card = cardinality_df[
    (cardinality_df['Categorías_Únicas'] > 10) & 
    (cardinality_df['Categorías_Únicas'] <= 100)
]
low_card = cardinality_df[cardinality_df['Categorías_Únicas'] <= 10]

print(f"\n📊 Resumen de Cardinalidad:")
print(f"   - Alta cardinalidad (>100): {len(high_card)} variables")
print(f"   - Media cardinalidad (10-100): {len(medium_card)} variables")
print(f"   - Baja cardinalidad (≤10): {len(low_card)} variables")

if len(high_card) > 0:
    print(f"\n💡 Variables de alta cardinalidad requerirán:")
    print(f"   - Target encoding o frequency encoding")
    print(f"   - Agrupación de categorías raras")
    print(f"   - Posible eliminación si no son informativas")

### 3.5 Análisis de Variables Numéricas

In [ ]:
# Estadísticas descriptivas de variables numéricas
numeric_features = train_data.select_dtypes(include=[np.number]).columns.tolist()
if 'id' in numeric_features:
    numeric_features.remove('id')

print("=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS - VARIABLES NUMÉRICAS")
print("=" * 80)
print(train_data[numeric_features].describe().round(2))

# Visualización de distribuciones
n_features = min(len(numeric_features), 9)
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.flatten()

for idx, col in enumerate(numeric_features[:n_features]):
    train_data[col].hist(bins=50, ax=axes[idx], edgecolor='black')
    axes[idx].set_title(f'Distribución: {col}', fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frecuencia')

# Ocultar ejes vacíos
for idx in range(n_features, 9):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.show()

## 4. Preprocesamiento y Feature Engineering

### 4.1 Preparación de Datos

**Estrategia de Preprocesamiento**:
1. Manejo de valores faltantes y ceros problemáticos
2. Extracción de características temporales
3. Encoding de variables categóricas
4. Feature engineering basado en conocimiento del dominio

In [ ]:
def preprocess_data(df, is_training=True):
    """
    Preprocesa el dataset aplicando limpieza y feature engineering.
    
    Args:
        df: DataFrame a procesar
        is_training: Si es True, incluye la variable objetivo
    
    Returns:
        DataFrame procesado
    """
    df = df.copy()
    
    print("Iniciando preprocesamiento...")
    
    # 1. Manejo de coordenadas geográficas
    print("  - Procesando coordenadas geográficas...")
    df['longitude'].replace(0, np.nan, inplace=True)
    df['latitude'].replace(0, np.nan, inplace=True)
    
    # Imputar con mediana
    df['longitude'].fillna(df['longitude'].median(), inplace=True)
    df['latitude'].fillna(df['latitude'].median(), inplace=True)
    
    # 2. Manejo de construction_year
    print("  - Procesando año de construcción...")
    df['construction_year'].replace(0, np.nan, inplace=True)
    df['construction_year'].fillna(df['construction_year'].median(), inplace=True)
    
    # Feature engineering: edad de la bomba
    current_year = 2013  # Año aproximado de los datos
    df['pump_age'] = current_year - df['construction_year']
    df['pump_age'] = df['pump_age'].clip(lower=0)  # No edades negativas
    
    # 3. Manejo de population
    print("  - Procesando población...")
    df['population'].replace(0, np.nan, inplace=True)
    df['population'].fillna(df['population'].median(), inplace=True)
    df['has_population'] = (df['population'] > 0).astype(int)
    
    # 4. Manejo de gps_height
    print("  - Procesando altura GPS...")
    df['gps_height'].replace(0, np.nan, inplace=True)
    df['gps_height'].fillna(df['gps_height'].median(), inplace=True)
    
    # 5. Feature engineering geográfico
    print("  - Creando features geográficas...")
    # Distancia al centro aproximado de Tanzania
    tanzania_center_lat = -6.369028
    tanzania_center_lon = 34.888822
    
    df['distance_to_center'] = np.sqrt(
        (df['latitude'] - tanzania_center_lat)**2 + 
        (df['longitude'] - tanzania_center_lon)**2
    )
    
    # 6. Extracción de fecha
    print("  - Procesando fecha de registro...")
    df['date_recorded'] = pd.to_datetime(df['date_recorded'])
    df['year_recorded'] = df['date_recorded'].dt.year
    df['month_recorded'] = df['date_recorded'].dt.month
    df['day_of_year'] = df['date_recorded'].dt.dayofyear
    
    # Eliminar la columna original de fecha
    df.drop('date_recorded', axis=1, inplace=True)
    
    # 7. Manejo de variables categóricas con valores vacíos
    print("  - Procesando variables categóricas...")
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    for col in categorical_cols:
        if col != 'status_group' or not is_training:
            # Reemplazar valores vacíos y NaN
            df[col].fillna('unknown', inplace=True)
            df[col].replace('', 'unknown', inplace=True)
            df[col].replace(' ', 'unknown', inplace=True)
    
    # 8. Reducción de cardinalidad para variables categóricas
    print("  - Reduciendo cardinalidad de variables categóricas...")
    high_cardinality_cols = ['installer', 'funder', 'wpt_name', 'subvillage', 'ward', 'scheme_name']
    
    for col in high_cardinality_cols:
        if col in df.columns:
            # Mantener top categorías, agrupar el resto como 'other'
            top_categories = df[col].value_counts().head(20).index
            df[col] = df[col].apply(
                lambda x: x if x in top_categories else 'other'
            )
    
    print("✓ Preprocesamiento completado")
    return df

# Aplicar preprocesamiento
print("\n" + "="*80)
print("APLICANDO PREPROCESAMIENTO")
print("="*80)

train_processed = preprocess_data(train_data, is_training=True)
test_processed = preprocess_data(test_values, is_training=False)

print(f"\nDimensiones finales:")
print(f"  - Train: {train_processed.shape}")
print(f"  - Test: {test_processed.shape}")

### 4.2 Encoding de Variables Categóricas

**Estrategia**: Usaremos Label Encoding para variables categóricas, ya que muchos algoritmos de árboles (Random Forest, XGBoost, LightGBM) lo manejan eficientemente.

In [ ]:
def encode_features(train_df, test_df, target_col='status_group'):
    """
    Codifica variables categóricas usando Label Encoding.
    
    Args:
        train_df: DataFrame de entrenamiento
        test_df: DataFrame de prueba
        target_col: Nombre de la columna objetivo
    
    Returns:
        train_encoded, test_encoded, label_encoders, target_encoder
    """
    train_encoded = train_df.copy()
    test_encoded = test_df.copy()
    
    # Guardar ID para submission
    test_ids = test_encoded['id'].copy()
    
    # Identificar columnas categóricas
    categorical_cols = train_encoded.select_dtypes(include=['object']).columns.tolist()
    if target_col in categorical_cols:
        categorical_cols.remove(target_col)
    
    print("=" * 80)
    print("CODIFICACIÓN DE VARIABLES CATEGÓRICAS")
    print("=" * 80)
    print(f"\nVariables a codificar: {len(categorical_cols)}")
    
    # Diccionario para guardar encoders
    label_encoders = {}
    
    # Codificar cada variable categórica
    for col in categorical_cols:
        le = LabelEncoder()
        
        # Combinar valores únicos de train y test
        combined_values = pd.concat([
            train_encoded[col],
            test_encoded[col]
        ]).unique()
        
        # Fit en valores combinados
        le.fit(combined_values)
        
        # Transform
        train_encoded[col] = le.transform(train_encoded[col])
        test_encoded[col] = le.transform(test_encoded[col])
        
        label_encoders[col] = le
    
    # Codificar variable objetivo
    target_encoder = LabelEncoder()
    train_encoded[target_col] = target_encoder.fit_transform(train_encoded[target_col])
    
    print(f"\n✓ Codificación completada")
    print(f"\nClases objetivo:")
    for idx, label in enumerate(target_encoder.classes_):
        print(f"  {idx}: {label}")
    
    return train_encoded, test_encoded, label_encoders, target_encoder, test_ids

# Aplicar encoding
train_encoded, test_encoded, label_encoders, target_encoder, test_ids = encode_features(
    train_processed, test_processed
)

### 4.3 Preparación de Datasets para Modelado

In [ ]:
# Separar características y objetivo
X = train_encoded.drop(['status_group', 'id'], axis=1)
y = train_encoded['status_group']

# Preparar test set
X_test_final = test_encoded.drop('id', axis=1)

# Asegurar mismas columnas
missing_cols = set(X.columns) - set(X_test_final.columns)
for col in missing_cols:
    X_test_final[col] = 0

X_test_final = X_test_final[X.columns]

print("=" * 80)
print("DATASETS FINALES PARA MODELADO")
print("=" * 80)
print(f"\nX (features): {X.shape}")
print(f"y (target): {y.shape}")
print(f"X_test_final: {X_test_final.shape}")
print(f"\nNúmero de características: {X.shape[1]}")
print(f"\nDistribución de clases:")
print(y.value_counts())

# Split train/validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nDivisión Train/Validation:")
print(f"  - X_train: {X_train.shape}")
print(f"  - X_val: {X_val.shape}")
print(f"  - y_train: {y_train.shape}")
print(f"  - y_val: {y_val.shape}")

## 5. Entrenamiento de Modelos

### 5.1 Configuración de Modelos

**Estrategia de Modelado**:
- Probar múltiples algoritmos (Random Forest, XGBoost, LightGBM)
- Usar class_weight para manejar desbalanceo
- Validación cruzada para robustez
- Comparar métricas de rendimiento

In [ ]:
# Calcular pesos de clase para manejar desbalanceo
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

print("=" * 80)
print("CONFIGURACIÓN DE MODELOS")
print("=" * 80)
print(f"\nPesos de clase (para manejar desbalanceo):")
for class_idx, weight in class_weight_dict.items():
    class_name = target_encoder.inverse_transform([class_idx])[0]
    print(f"  Clase {class_idx} ({class_name}): {weight:.3f}")

# Definir modelos
models = {
    'Random Forest': RandomForestClassifier(
        n_estimators=200,
        max_depth=20,
        min_samples_split=5,
        min_samples_leaf=2,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ),
    'XGBoost': XGBClassifier(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        eval_metric='mlogloss'
    ),
    'LightGBM': LGBMClassifier(
        n_estimators=200,
        max_depth=10,
        learning_rate=0.1,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )
}

print(f"\n✓ Modelos configurados: {len(models)}")

### 5.2 Entrenamiento y Evaluación de Modelos

In [ ]:
print("=" * 80)
print("ENTRENAMIENTO DE MODELOS")
print("=" * 80)

results = {}
trained_models = {}

for name, model in models.items():
    print(f"\n{'='*80}")
    print(f"Entrenando: {name}")
    print(f"{'='*80}")
    
    # Entrenar modelo
    model.fit(X_train, y_train)
    trained_models[name] = model
    
    # Predicciones
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    
    # Métricas
    train_accuracy = accuracy_score(y_train, y_pred_train)
    val_accuracy = accuracy_score(y_val, y_pred_val)
    
    print(f"\n📊 Resultados:")
    print(f"  - Accuracy (Train): {train_accuracy:.4f}")
    print(f"  - Accuracy (Validation): {val_accuracy:.4f}")
    print(f"  - Diferencia: {abs(train_accuracy - val_accuracy):.4f}")
    
    # Reporte de clasificación
    print(f"\n📋 Reporte de Clasificación (Validation):")
    print(classification_report(
        y_val, 
        y_pred_val,
        target_names=target_encoder.classes_,
        digits=4
    ))
    
    # Guardar resultados
    results[name] = {
        'train_accuracy': train_accuracy,
        'val_accuracy': val_accuracy,
        'y_pred_val': y_pred_val
    }
    
    # Matriz de confusión
    cm = confusion_matrix(y_val, y_pred_val)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=target_encoder.classes_,
                yticklabels=target_encoder.classes_)
    plt.title(f'Matriz de Confusión - {name}', fontsize=14, fontweight='bold')
    plt.ylabel('Valor Real')
    plt.xlabel('Predicción')
    plt.tight_layout()
    plt.show()

print(f"\n\n{'='*80}")
print("RESUMEN DE RESULTADOS")
print(f"{'='*80}")

results_df = pd.DataFrame({
    'Modelo': results.keys(),
    'Train Accuracy': [r['train_accuracy'] for r in results.values()],
    'Validation Accuracy': [r['val_accuracy'] for r in results.values()]
})
results_df = results_df.sort_values('Validation Accuracy', ascending=False)
print(results_df.to_string(index=False))

best_model_name = results_df.iloc[0]['Modelo']
best_model = trained_models[best_model_name]
best_accuracy = results_df.iloc[0]['Validation Accuracy']

print(f"\n🏆 Mejor Modelo: {best_model_name}")
print(f"📈 Accuracy en Validación: {best_accuracy:.4f}")

### 5.3 Análisis de Importancia de Features

**Valor de Negocio**: Identificar qué factores son más importantes para predecir el estado de las bombas ayuda a:
- Priorizar la recolección de datos
- Enfocar recursos de mantenimiento
- Mejorar la toma de decisiones

In [ ]:
# Obtener importancia de features del mejor modelo
if hasattr(best_model, 'feature_importances_'):
    print("=" * 80)
    print(f"IMPORTANCIA DE CARACTERÍSTICAS - {best_model_name}")
    print("=" * 80)
    
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nTop 20 Características Más Importantes:")
    print(feature_importance.head(20).to_string(index=False))
    
    # Visualización
    plt.figure(figsize=(12, 8))
    top_features = feature_importance.head(20)
    plt.barh(range(len(top_features)), top_features['Importance'])
    plt.yticks(range(len(top_features)), top_features['Feature'])
    plt.xlabel('Importancia')
    plt.title('Top 20 Características Más Importantes', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Insights de Negocio:")
    print("   Las características más importantes pueden guiar:")
    print("   - Inversiones en infraestructura")
    print("   - Protocolos de mantenimiento")
    print("   - Recolección de datos críticos")

## 6. Predicciones Finales y Submission

### 6.1 Generar Predicciones para el Test Set

In [ ]:
print("=" * 80)
print("GENERANDO PREDICCIONES FINALES")
print("=" * 80)

# Reentrenar el mejor modelo con todos los datos de entrenamiento
print(f"\nReentrenando {best_model_name} con todos los datos de entrenamiento...")
best_model.fit(X, y)

# Predicciones en test set
y_pred_test = best_model.predict(X_test_final)

# Convertir predicciones numéricas a labels originales
y_pred_test_labels = target_encoder.inverse_transform(y_pred_test)

print(f"\n✓ Predicciones generadas: {len(y_pred_test_labels):,}")
print(f"\nDistribución de predicciones:")
pred_counts = pd.Series(y_pred_test_labels).value_counts()
for status, count in pred_counts.items():
    percentage = (count / len(y_pred_test_labels)) * 100
    print(f"  - {status}: {count:,} ({percentage:.2f}%)")

### 6.2 Crear Archivo de Submission

In [ ]:
# Crear DataFrame de submission
submission = pd.DataFrame({
    'id': test_ids,
    'status_group': y_pred_test_labels
})

# Guardar archivo
submission_filename = 'submission.csv'
submission.to_csv(submission_filename, index=False)

print("=" * 80)
print("ARCHIVO DE SUBMISSION CREADO")
print("=" * 80)
print(f"\n✓ Archivo guardado como: {submission_filename}")
print(f"\nPrimeras filas del archivo de submission:")
print(submission.head(10))
print(f"\nÚltimas filas del archivo de submission:")
print(submission.tail(10))

print(f"\n\n{'='*80}")
print("INSTRUCCIONES PARA SUBIR LA SUBMISSION")
print(f"{'='*80}")
print("\n1. Ir a: https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/submissions/")
print("2. Hacer clic en 'Submit Predictions'")
print(f"3. Subir el archivo: {submission_filename}")
print("4. El score aparecerá en el leaderboard")
print("\n💡 El score oficial será la métrica 'Classification Rate' en el leaderboard")

## 7. Conclusiones y Recomendaciones de Negocio

### 7.1 Resumen Ejecutivo

In [ ]:
print("=" * 80)
print("RESUMEN EJECUTIVO DEL PROYECTO")
print("=" * 80)

print(f"\n📊 MÉTRICAS DEL MODELO")
print(f"{'-'*80}")
print(f"Mejor Modelo: {best_model_name}")
print(f"Accuracy en Validación: {best_accuracy:.2%}")
print(f"Total de características usadas: {X.shape[1]}")
print(f"Total de bombas en entrenamiento: {len(train_data):,}")
print(f"Total de predicciones generadas: {len(submission):,}")

print(f"\n🎯 DISTRIBUCIÓN DE CLASES EN PREDICCIONES")
print(f"{'-'*80}")
for status, count in pred_counts.items():
    percentage = (count / len(y_pred_test_labels)) * 100
    print(f"{status:30s}: {count:6,} ({percentage:5.2f}%)")

print(f"\n💡 INSIGHTS CLAVE PARA EL NEGOCIO")
print(f"{'-'*80}")
print("\n1. CALIDAD DE LOS DATOS")
print("   - Se detectaron valores faltantes y ceros en variables clave (coordenadas, año)")
print("   - Recomendación: Mejorar procesos de recolección de datos en campo")

print("\n2. FACTORES CRÍTICOS DE FALLA")
print("   - Las características más importantes incluyen:")
if hasattr(best_model, 'feature_importances_'):
    top_3_features = feature_importance.head(3)['Feature'].tolist()
    for feat in top_3_features:
        print(f"     • {feat}")
print("   - Recomendación: Monitorear estos factores proactivamente")

print("\n3. DESBALANCEO DE CLASES")
print("   - 'functional needs repair' es la clase minoritaria")
print("   - Recomendación: Establecer alertas tempranas para bombas en esta categoría")

print("\n4. OPORTUNIDADES DE MEJORA")
print("   - Recolectar más datos de bombas que necesitan reparación")
print("   - Implementar sistema de monitoreo en tiempo real")
print("   - Desarrollar estrategia de mantenimiento preventivo basada en predicciones")

print(f"\n\n{'='*80}")
print("PRÓXIMOS PASOS")
print(f"{'='*80}")
print("\n1. ✅ Subir submission.csv al concurso de DrivenData")
print("2. 📝 Registrar el score oficial obtenido")
print("3. 🔄 Iterar mejorando el modelo si es necesario:")
print("   - Probar feature engineering adicional")
print("   - Optimizar hiperparámetros con GridSearch/RandomSearch")
print("   - Considerar ensemble de modelos")
print("4. 🚀 Implementar modelo en producción")
print("5. 📊 Establecer métricas de negocio para monitoreo continuo")

### 7.2 Registro del Score del Concurso

**IMPORTANTE**: Después de subir la submission al concurso, registra aquí el score oficial obtenido.

In [ ]:
# COMPLETAR DESPUÉS DE SUBIR AL CONCURSO
print("=" * 80)
print("SCORE OFICIAL DEL CONCURSO")
print("=" * 80)
print("\n⚠️  INSTRUCCIONES:")
print("   1. Sube el archivo 'submission.csv' al concurso")
print("   2. Anota el score (Classification Rate) que aparece en el leaderboard")
print("   3. Actualiza la variable 'competition_score' a continuación")
print("\n" + "-"*80)

# ACTUALIZAR ESTA VARIABLE CON EL SCORE REAL
competition_score = None  # Ejemplo: 0.8234

if competition_score is not None:
    print(f"\n🏆 Score Oficial del Concurso: {competition_score:.4f}")
    print(f"   Accuracy en Validación Local: {best_accuracy:.4f}")
    print(f"   Diferencia: {abs(competition_score - best_accuracy):.4f}")
    
    if abs(competition_score - best_accuracy) < 0.02:
        print("\n✅ El modelo generaliza bien - scores consistentes")
    else:
        print("\n⚠️  Diferencia significativa - posible overfitting o diferencias en datos")
else:
    print("\n❌ Score aún no registrado")
    print("   Por favor, sube la submission y actualiza la variable 'competition_score'")

## 8. Mejoras Futuras (Opcional)

Si el score inicial no es satisfactorio, considera estas mejoras:

### 8.1 Feature Engineering Avanzado
- Crear interacciones entre features importantes
- Agregar features geoespaciales más complejas
- Aplicar transformaciones no lineales

### 8.2 Optimización de Hiperparámetros
- GridSearchCV o RandomizedSearchCV
- Bayesian Optimization
- Validación cruzada más robusta

### 8.3 Ensemble Methods
- Voting Classifier combinando múltiples modelos
- Stacking
- Blending

### 8.4 Técnicas de Balanceo Más Sofisticadas
- ADASYN
- Diferentes estrategias de SMOTE
- Cost-sensitive learning

---

## Fin del Notebook

**Autor**: [Tu Nombre]

**Fecha**: [Fecha]

**Competición**: Pump it Up - DrivenData.org

---